# Einführung in SQL - Vorbereitungsauftrag - Teil 2

In [ ]:
from util.sqlite_util import *
# Test-Datenbank öffnen
path = get_connection_path("chinook.sqlite")
connection = create_connection(path)

Connection to SQLite DB 2.6.0 (./data/chinook.sqlite) successful


## Tabellen verbinden:  INNER JOIN  (ON oder USING)

Mit *Joins* können mehrere Tabellen einer Datenbank miteinander verbunden werden. *Joins* gehören zu den Hauptfunktionen von relationalen Datenbanken. Sie sind aber nicht immer intuitiv zu verstehen, weshalb wir sie im Präsenzunterricht vertieft anschauen werden. Vorerst geht es hier darum, die grundsätzliche Funktionsweise zu verstehen. 

Am meisten gebraucht werden sogenannte INNER JOINs.
* **(INNER) JOIN**: Enthält alle Datensätze, die korrespondierende Einträge in beiden Tabellen haben. Das Keyword INNER ist optional, sollte aber immer ausgeschrieben werden (siehe oben).
  ![Inner Join](./img/img_innerjoin.gif)


Wir verwenden die tabellen `artists` und `albums`, wobei jedem *Album* genau ein *Artist* zugewiesen wird (Attribut `albums.ArtistsId` zeigt auf `artists.ArtistId`). Ein *Artist* kann so gar kein, ein oder mehrere *Alben* geschrieben haben. 

![Artists_Albums](./img/artists_albums.png)



In [ ]:
# Der INNER JOIN berücksichtig nur Einträge, die in beiden Tabellen vorkommen und miteinander verknüpft sind.
# 'albums' ohne 'artist' oder 'artists' ohne 'album' werden ignoriert. 
stmt = """SELECT 
    artists.Name,
    albums.Title
FROM 
    albums 
    INNER JOIN 
    artists
        ON artists.ArtistId = albums.ArtistId;"""
cur = execute_query(connection, stmt)
print_results(cur)


['Name', 'Title']
('AC/DC', 'For Those About To Rock We Salute You')
('Accept', 'Balls to the Wall')
('Accept', 'Restless and Wild')
('AC/DC', 'Let There Be Rock')
('Aerosmith', 'Big Ones')
('Alanis Morissette', 'Jagged Little Pill')
('Alice In Chains', 'Facelift')
('Antônio Carlos Jobim', 'Warner 25 Anos')
('Apocalyptica', 'Plays Metallica By Four Cellos')
('Audioslave', 'Audioslave')


In [ ]:
# Wir können Aliasse verwenden, um die Query zu vereinfachen: al für Album, ar für Artist.
# Dass l für left und r für right steht, ist ein didaktisch motivierter 'Zufall' ;-).
stmt = """SELECT 
    al.Title,
    ar.Name
FROM 
    albums al 
    INNER JOIN
    artists ar
        ON ar.ArtistId = al.ArtistId;"""
cur = execute_query(connection, stmt)
print_results(cur)


['Title', 'Name']
('For Those About To Rock We Salute You', 'AC/DC')
('Balls to the Wall', 'Accept')
('Restless and Wild', 'Accept')
('Let There Be Rock', 'AC/DC')
('Big Ones', 'Aerosmith')
('Jagged Little Pill', 'Alanis Morissette')
('Facelift', 'Alice In Chains')
('Warner 25 Anos', 'Antônio Carlos Jobim')
('Plays Metallica By Four Cellos', 'Apocalyptica')
('Audioslave', 'Audioslave')


In [ ]:
# Da ArtistId in beiden Tabellen gleich heisst, können wir USING verwenden.
stmt = """SELECT 
    al.Title,
    ar.Name
FROM 
    albums al 
    INNER JOIN
    artists ar
        USING(ArtistId);"""
cur = execute_query(connection, stmt)
print_results(cur)


['Title', 'Name']
('For Those About To Rock We Salute You', 'AC/DC')
('Balls to the Wall', 'Accept')
('Restless and Wild', 'Accept')
('Let There Be Rock', 'AC/DC')
('Big Ones', 'Aerosmith')
('Jagged Little Pill', 'Alanis Morissette')
('Facelift', 'Alice In Chains')
('Warner 25 Anos', 'Antônio Carlos Jobim')
('Plays Metallica By Four Cellos', 'Apocalyptica')
('Audioslave', 'Audioslave')


### Aufgabe #5

In [ ]:
# Aufgabe #5: Zeige für alle 'customers' die 'invoices' geordnet nach absteigendem Datum.


## Tabellen verbinden:  OUTER und CROSS JOINs

Neben dem INNER JOIN unterscheidet man weitere Arten von joins. 
* **LEFT (OUTER) JOIN**: Enthält alle Datensätze der linken Tabelle und die korrespondierenden Datensätze der rechten Tabelle. Das Keyword OUTER wird im Normalfall weggelassen.
  ![Left Join](./img/img_leftjoin.gif)
* **RIGHT (OUTER) JOIN**: Enthält alle Datensätze der rechten Tabelle und die korrespondierenden Datensätze der linken Tabelle. RIGHT JOINS werden von sqlite nicht unterstützt können aber problemlos durch einen LEFT JOIN implementiert werden, indem die Tabellen getauscht werden.
    ![Rigth Join](./img/img_rightjoin.gif)
* **FULL (OUTER) JOIN**: Enthält alle Datensätze der linken und rechten Datensätze und verknüft diejenigen mit passenden JOIN-Bedingungen. FULL JOINS werden von sqlite nicht unterstützt können aber mit einer UNION emuliert werden: `a LEFT JOIN b UNION b LEFT JOIN a`. Dies wird hier nicht weiter ausgeführt, da FULL JOINS nur selten benutzt werden.
  ![Full Join](./img/img_fulljoin.gif)
* **CROSS JOIN** Bildet das Kreuzprodukt zweier Tabellen. Dabei wird jeder Datensatz von Tabelle a mit jedem Datensatz von Tabelle b verknüpft. Ursprünglich unterstütze SQL nur Cross Joins. Mit geeigneten WHERE-Clauses können alle anderen Joins emuliert werden. Seit SQL-92 ist der CROSS JOIN aber nur noch für Spezialfälle notwendig und wird deshalb hier nicht weiter betrachtet.


Quelle: https://www.w3schools.com/sql/sql_join.asp
Je nach DBMS werden weitere joins unterstützt.

In [ ]:
# Betrachten wir nun also exemplarisch einen left join
# Anmerkung. Hier haben ar und al die Plätze getauscht. Eigentlich handelt es sich also um 
# einen right join, der in einen left join umgewandelt wurde.
stmt = """SELECT
   ar.Name, 
   al.Title
FROM
   artists ar
   LEFT JOIN 
   albums al
USING (ArtistId)
ORDER BY Name;"""
cur = execute_query(connection, stmt)
rows = cur.fetchall()
print(rows)

# Jetzt werden auch Künstler ohne zugehöriges Album aufgelistet. Z.B. "A Cor Do Som".

[('A Cor Do Som', None), ('AC/DC', 'For Those About To Rock We Salute You'), ('AC/DC', 'Let There Be Rock'), ('Aaron Copland & London Symphony Orchestra', 'A Copland Celebration, Vol. I'), ('Aaron Goldberg', 'Worlds'), ('Academy of St. Martin in the Fields & Sir Neville Marriner', 'The World of Classical Favourites'), ('Academy of St. Martin in the Fields Chamber Ensemble & Sir Neville Marriner', 'Sir Neville Marriner: A Celebration'), ('Academy of St. Martin in the Fields, John Birch, Sir Neville Marriner & Sylvia McNair', 'Fauré: Requiem, Ravel: Pavane & Others'), ('Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart', 'Bach: Orchestral Suites Nos. 1 - 4'), ('Academy of St. Martin in the Fields, Sir Neville Marriner & William Bennett', None), ('Accept', 'Balls to the Wall'), ('Accept', 'Restless and Wild'), ('Adrian Leaper & Doreen de Feis', 'Górecki: Symphony No. 3'), ('Aerosmith', 'Big Ones'), ("Aerosmith & Sierra Leone's Refugee Allstars", None), ('Aisha Duo'

### Aufgabe #6

In [ ]:
# Aufgabe #6: Finde alle Künstler, die kein Album zugeweisen haben. Tipp: WHERE-Clause verwenden.


## Daten gruppieren: GROUP BY

Mit `SELECT column_list FROM table GROUP BY column_list` können mehrere Datensätze zu einem einzigen Datensatz zuammengefasst bzw. gruppiert werden. Typischerweise werden dabei oft Aggregationsfunktionen wie AVG, SUM, etc auf einzelne Kolonnen angewendet.

Im Folgenden benutzen wir die Tabelle `tracks` aus der Beispieldatenbank.

![Tracks und Albums](./img/tracks_albums.png)

In [ ]:
# Als erstes zählen wir die Tracks pro Album
stmt = """SELECT
	albumid,
	COUNT(trackid)
FROM
	tracks
GROUP BY
	albumid;"""
cur = execute_query(connection, stmt)
rows = cur.fetchall()
print(rows)

[(1, 10), (2, 1), (3, 3), (4, 8), (5, 15), (6, 13), (7, 12), (8, 14), (9, 8), (10, 14), (11, 12), (12, 12), (13, 8), (14, 13), (15, 5), (16, 7), (17, 10), (18, 17), (19, 11), (20, 11), (21, 18), (22, 3), (23, 34), (24, 23), (25, 13), (26, 17), (27, 14), (28, 10), (29, 14), (30, 14), (31, 9), (32, 14), (33, 17), (34, 17), (35, 11), (36, 17), (37, 20), (38, 12), (39, 21), (40, 12), (41, 14), (42, 14), (43, 7), (44, 6), (45, 14), (46, 13), (47, 14), (48, 13), (49, 10), (50, 4), (51, 22), (52, 15), (53, 14), (54, 20), (55, 20), (56, 15), (57, 15), (58, 9), (59, 7), (60, 7), (61, 11), (62, 7), (63, 12), (64, 9), (65, 9), (66, 10), (67, 16), (68, 9), (69, 13), (70, 13), (71, 14), (72, 18), (73, 30), (74, 12), (75, 14), (76, 15), (77, 11), (78, 14), (79, 10), (80, 10), (81, 11), (82, 13), (83, 24), (84, 16), (85, 14), (86, 15), (87, 3), (88, 12), (89, 13), (90, 12), (91, 16), (92, 14), (93, 13), (94, 11), (95, 12), (96, 11), (97, 10), (98, 11), (99, 12), (100, 9), (101, 10), (102, 18), (103, 

In [ ]:
# Wir können die Alben auch nach Anzahl tracks sortieren (ORDER BY)
stmt = """SELECT
	albumid,
	COUNT(trackid)
FROM
	tracks
GROUP BY
	albumid
ORDER BY COUNT(trackid) DESC;"""
cur = execute_query(connection, stmt)
rows = cur.fetchall()
print(rows)

[(141, 57), (23, 34), (73, 30), (229, 26), (230, 25), (251, 25), (83, 24), (231, 24), (253, 24), (24, 23), (228, 23), (255, 23), (51, 22), (224, 22), (250, 22), (39, 21), (167, 21), (37, 20), (54, 20), (55, 20), (115, 20), (221, 20), (227, 19), (248, 19), (258, 19), (21, 18), (72, 18), (102, 18), (145, 18), (146, 18), (202, 18), (211, 18), (213, 18), (18, 17), (26, 17), (33, 17), (34, 17), (36, 17), (120, 17), (162, 17), (163, 17), (185, 17), (193, 17), (203, 17), (237, 17), (243, 17), (259, 17), (261, 17), (67, 16), (84, 16), (91, 16), (125, 16), (140, 16), (149, 16), (169, 16), (178, 16), (184, 16), (190, 16), (194, 16), (201, 16), (225, 16), (5, 15), (52, 15), (56, 15), (57, 15), (76, 15), (86, 15), (126, 15), (139, 15), (160, 15), (165, 15), (180, 15), (195, 15), (218, 15), (219, 15), (222, 15), (223, 15), (234, 15), (247, 15), (8, 14), (10, 14), (27, 14), (29, 14), (30, 14), (32, 14), (41, 14), (42, 14), (45, 14), (47, 14), (53, 14), (71, 14), (75, 14), (78, 14), (85, 14), (92, 14

In [ ]:
# Dabei können auch Daten von mehreren Tabellen zuerst verbunden (JOIN) und dann gruppiert werden. So erhalten wir den Album-Titel im Resultat.
stmt = """SELECT
	t.albumid,
	a.title,
	COUNT(t.trackid)
FROM
	tracks t
INNER JOIN albums a USING(albumid)
GROUP BY
	t.albumid
ORDER BY COUNT(t.trackid) DESC;"""
cur = execute_query(connection, stmt)
rows = cur.fetchall()
print(rows)

[(141, 'Greatest Hits', 57), (23, 'Minha Historia', 34), (73, 'Unplugged', 30), (229, 'Lost, Season 3', 26), (230, 'Lost, Season 1', 25), (251, 'The Office, Season 3', 25), (83, 'My Way: The Best Of Frank Sinatra [Disc 1]', 24), (231, 'Lost, Season 2', 24), (253, 'Battlestar Galactica (Classic), Season 1', 24), (24, 'Afrociberdelia', 23), (228, 'Heroes, Season 1', 23), (255, 'Instant Karma: The Amnesty International Campaign to Save Darfur', 23), (51, "Up An' Atom", 22), (224, 'Acústico', 22), (250, 'The Office, Season 2', 22), (39, 'International Superhits', 21), (167, 'Acústico MTV', 21), (37, 'Greatest Kiss', 20), (54, 'Chronicle, Vol. 1', 20), (55, 'Chronicle, Vol. 2', 20), (115, 'Sex Machine', 20), (221, 'My Generation - The Very Best Of The Who', 20), (227, 'Battlestar Galactica, Season 3', 19), (248, 'Ao Vivo [IMPORT]', 19), (258, 'House of Pain', 19), (21, 'Prenda Minha', 18), (72, 'The Cream Of Clapton', 18), (102, 'Live After Death', 18), (145, 'Barulhinho Bom', 18), (146, 'S

### Aufgabe #7

In [ ]:
# Aufgabe #7: Öffne https://www.sqlitetutorial.net/sqlite-group-by/ und wähle dir ein Beispiel nach den Kapitel "SQLite GROUP BY with HAVING clause" aus. 
# Implementiere es und experimentiere damit. 


### Aufgabe #8
Um die Qualtität der Übung für zukünftige Klassen zu verbessern bin ich sehr dankbar für Feedback zu diesen Übungen. Die folgende Umfrage besteht nur aus 5 Fragen und braucht weniger als 3 Minuten Zeit: https://forms.gle/WfZiyUjXvbvYmBnY7
Vielen Dank! 

Wem es jetzt den Ärmel reingezogen hat, darf gerne an https://www.sqlitetutorial.net/ weiterarbeiten. Als Vorbereitung für den Präsenzunterricht 
reicht es aber für den Moment. 

Bei Fragen oder Anmerkungen: marco.soldati@fhnw.ch oder via Teams. Ich freue mich auf den ersten Halbtag mit euch. Marco Soldati

# Lösungen

In [ ]:
# Aufgabe #5: 
# Zuerst zeigen wir die Struktur der Tabelle an. Die Hilfsmethoden sind in der Datei ./util/sqlite_util.py gespeichert.

print(show_tables(connection))
print(desc_table(connection, "invoices"))

['albums', 'artists', 'customers', 'employees', 'genres', 'invoices', 'invoice_items', 'media_types', 'playlists', 'playlist_track', 'tracks']
CREATE TABLE "invoices"
(
    [InvoiceId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    [CustomerId] INTEGER  NOT NULL,
    [InvoiceDate] DATETIME  NOT NULL,
    [BillingAddress] NVARCHAR(70),
    [BillingCity] NVARCHAR(40),
    [BillingState] NVARCHAR(40),
    [BillingCountry] NVARCHAR(40),
    [BillingPostalCode] NVARCHAR(10),
    [Total] NUMERIC(10,2)  NOT NULL,
    FOREIGN KEY ([CustomerId]) REFERENCES "customers" ([CustomerId]) 
		ON DELETE NO ACTION ON UPDATE NO ACTION
)


In [ ]:
# Aufgabe #5:
stmt = """SELECT 
    c.firstname, c.lastname, i.total, DATE(i.invoicedate)
FROM 
    customers c 
    LEFT JOIN
    invoices i
        USING(CustomerId)
WHERE
    i.billingaddress = c.address
ORDER BY 
    i.invoicedate desc;"""
cur = execute_query(connection, stmt)
rows = cur.fetchall()
print(rows)

[('Manoj', 'Pareek', 1.99, '2013-12-22'), ('Terhi', 'Hämäläinen', 13.86, '2013-12-14'), ('Madalena', 'Sampaio', 8.91, '2013-12-09'), ('Robert', 'Brown', 5.94, '2013-12-06'), ('Victor', 'Stevens', 3.96, '2013-12-05'), ('Kathy', 'Chase', 1.98, '2013-12-04'), ('John', 'Gordon', 1.98, '2013-12-04'), ('Dan', 'Miller', 0.99, '2013-11-21'), ('Helena', 'Holý', 25.86, '2013-11-13'), ('Diego', 'Gutiérrez', 8.91, '2013-11-08'), ('Enrique', 'Muñoz', 5.94, '2013-11-05'), ('Hugh', "O'Reilly", 3.96, '2013-11-04'), ('Wyatt', 'Girard', 1.98, '2013-11-03'), ('Terhi', 'Hämäläinen', 1.98, '2013-11-03'), ('Marc', 'Dubois', 0.99, '2013-10-21'), ('Patrick', 'Gray', 13.86, '2013-10-13'), ('Michelle', 'Brooks', 8.91, '2013-10-08'), ('Roberto', 'Almeida', 5.94, '2013-10-05'), ('Daan', 'Peeters', 3.96, '2013-10-04'), ('Bjørn', 'Hansen', 1.98, '2013-10-03'), ('Helena', 'Holý', 1.98, '2013-10-03'), ('François', 'Tremblay', 0.99, '2013-09-20'), ('Johannes', 'Van der Berg', 13.86, '2013-09-12'), ('Camille', 'Bernard

In [ ]:
# Aufgabe #6
stmt = """SELECT
   ar.Name, 
   al.Title
FROM
   artists ar
   LEFT JOIN 
   albums al
USING (ArtistId)
WHERE Title IS NULL   
ORDER BY Name;"""
cur = execute_query(connection, stmt)
rows = cur.fetchall()
print(rows)

[('A Cor Do Som', None), ('Academy of St. Martin in the Fields, Sir Neville Marriner & William Bennett', None), ("Aerosmith & Sierra Leone's Refugee Allstars", None), ('Avril Lavigne', None), ('Azymuth', None), ('Baby Consuelo', None), ('Banda Black Rio', None), ('Barão Vermelho', None), ('Bebel Gilberto', None), ('Ben Harper', None), ('Big & Rich', None), ('Black Eyed Peas', None), ('Charlie Brown Jr.', None), ('Christina Aguilera featuring BigElf', None), ('Corinne Bailey Rae', None), ('DJ Dolores & Orchestra Santa Massa', None), ('Dhani Harrison & Jakob Dylan', None), ('Edson, DJ Marky & DJ Patife Featuring Fernanda Porto', None), ('Fernanda Porto', None), ('Gustavo & Andres Veiga & Salazar', None), ('Hermeto Pascoal', None), ('Instituto', None), ('Jack Johnson', None), ("Jack's Mannequin & Mick Fleetwood", None), ('Jackson Browne', None), ('Jaguares', None), ('Jorge Vercilo', None), ('João Gilberto', None), ('Kid Abelha', None), ('Los Hermanos', None), ('Los Lonely Boys', None), ('

In [ ]:
# Aufgabe #7: Keine Lösung, da die Aufgabe generisch gestellt wurde

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7b26de29-b5f7-4099-8d22-b8e52b4135d4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>